In [1]:
import warnings
import os
from glob import glob
from pathlib import Path
import operator

import pandas as pd
import polars as pl
import polars.selectors as cs
from sklearn import preprocessing
from category_encoders import OneHotEncoder
from category_encoders.target_encoder import TargetEncoder

pd.set_option('display.float_format',lambda x : '%.6f' % x)
warnings.filterwarnings('ignore')

In [2]:
def set_table_dtypes(df):
    for col in df.columns:
        if col in ["case_id", "WEEK_NUM", "num_group1", "num_group2", "target"]:
            df = df.with_columns(pl.col(col).cast(pl.Int64))
        elif col in ["date_decision"]:
            df = df.with_columns(pl.col(col).cast(pl.Date))
        elif col[-1] in ("P", "A"):
            df = df.with_columns(pl.col(col).cast(pl.Float64))
        elif col[-1] in ("M",):
            df = df.with_columns(pl.col(col).cast(pl.String))
        elif col[-1] in ("D",):
            df = df.with_columns(pl.col(col).cast(pl.Date))

    return df

In [3]:
def read_file(path, depth=None):
    df = pl.read_parquet(path)
    df = df.pipe(set_table_dtypes)
    return df

def read_files(regex_path, depth=None):
    chunks = []
    for path in glob(str(regex_path)):
        chunks.append(pl.read_parquet(path).pipe(set_table_dtypes))

    df = pl.concat(chunks, how="vertical_relaxed")
    return df

In [4]:
train_dir_path = "/Users/sophie/workspace/data/kaggle/home-credit-credit-risk-model-stability/parquet_files/train"
test_dir_path = "/Users/sophie/workspace/data/kaggle/home-credit-credit-risk-model-stability/parquet_files/test"
train_file = 'train_deposit_1.parquet'
test_file = 'test_deposit_1.parquet'
train_file_path = Path(os.path.join(train_dir_path, train_file))
test_file_path = Path(os.path.join(test_dir_path, test_file))

target_file = 'train_base.parquet'
target_file_path = Path(os.path.join(train_dir_path, target_file))
target_file_test = 'test_base.parquet'
target_file_test_path = Path(os.path.join(test_dir_path, target_file_test))

In [5]:
df_base_train = read_file(target_file_path)
df_base_train = df_base_train.with_columns(IS_TRAIN = pl.lit(1))
df_train = read_file(train_file_path)
df_train = df_base_train.join(df_train, how="left", on="case_id")

In [6]:
df_base_test = read_file(target_file_test_path)
df_base_test = df_base_test.with_columns(target = None)
df_base_test = df_base_test.with_columns(IS_TRAIN = pl.lit(0))
df_test = read_file(test_file_path)
df_test = df_base_test.join(df_test, how="left", on="case_id")

In [7]:
data = pl.concat([df_train, df_test], how="vertical_relaxed")

In [8]:
data_base = pl.concat([df_base_train, df_base_test], how="vertical_relaxed")

In [28]:
del df_train, df_test

In [21]:
data.head(50)

case_id,date_decision,MONTH,WEEK_NUM,target,IS_TRAIN,amount_416A,contractenddate_991D,num_group1,openingdate_313D,end_to_open_date
i64,date,i64,i64,i64,i32,f64,date,i64,date,i64
0,2019-01-03,201901,0,0,1,null,null,null,null,null
1,2019-01-03,201901,0,0,1,null,null,null,null,null
2,2019-01-04,201901,0,0,1,null,null,null,null,null
3,2019-01-03,201901,0,0,1,null,null,null,null,null
4,2019-01-04,201901,0,1,1,null,null,null,null,null
5,2019-01-02,201901,0,0,1,null,null,null,null,null
6,2019-01-03,201901,0,0,1,null,null,null,null,null
7,2019-01-03,201901,0,0,1,null,null,null,null,null
8,2019-01-03,201901,0,0,1,null,null,null,null,null


In [26]:
# df_train.filter(pl.col("case_id") == 57633)
# data.select(['case_id', 'num_group1', 'target']).filter(pl.col('num_group1').is_not_null())
data.filter(pl.col('num_group1').is_not_null())
# df_train.shape

case_id,date_decision,MONTH,WEEK_NUM,target,IS_TRAIN,amount_416A,contractenddate_991D,num_group1,openingdate_313D,end_to_open_date,contractenddate_991D_minus_openingdate_313D_P
i64,date,i64,i64,i64,i32,f64,date,i64,date,i64,i64
225,2019-01-04,201901,0,0,1,0.0,null,0,2016-08-16,null,null
331,2019-01-05,201901,0,0,1,260.374,2018-03-18,0,2015-03-19,1095,1095
358,2019-01-05,201901,0,0,1,0.0,null,0,2014-09-02,null,null
390,2019-01-07,201901,0,0,1,211748.53,2017-07-22,0,2014-07-23,1095,1095
390,2019-01-07,201901,0,0,1,203.602,2017-09-30,1,2015-10-01,730,730
390,2019-01-07,201901,0,0,1,223.68001,null,2,2016-06-08,null,null
445,2019-01-08,201901,1,0,1,0.0,null,0,2015-04-27,null,null
445,2019-01-08,201901,1,0,1,0.0,2017-07-31,1,2015-08-01,730,730
445,2019-01-08,201901,1,0,1,0.0,2018-02-08,2,2015-12-10,791,791


In [23]:
data = data.with_columns((pl.col('contractenddate_991D') - pl.col("openingdate_313D")).alias("contractenddate_991D_minus_openingdate_313D_P"))
data = data.with_columns(pl.col("contractenddate_991D_minus_openingdate_313D_P").dt.total_days())

In [25]:
data = data.sort('case_id', 'num_group1')

In [27]:
class Aggregator:
    @staticmethod
    def num_expr(df):
        cols = [col for col in df.columns if col[-1] in ("P", "A")]

        expr_num = []
        # expr_num.extend([pl.max(col).alias(f"max_{col}") for col in cols])
        # expr_num.extend([pl.min(col).alias(f"min_{col}") for col in cols])
        expr_num.extend([pl.mean(col).alias(f"mean_{col}") for col in cols])
        # expr_num.extend([pl.count(col).alias(f"count_{col}") for col in cols])
        expr_num.extend([pl.var(col).alias(f"var_{col}") for col in cols])
        expr_num.extend([pl.last(col).alias(f"last_{col}") for col in cols])
        expr_num.extend([pl.first(col).alias(f"first_{col}") for col in cols])

        return expr_num

    @staticmethod
    def date_expr(df):
        cols = [col for col in df.columns if col[-1] in ("D",)]

        expr_date = []
        expr_date.extend([pl.max(col).alias(f"max_{col}") for col in cols])
        expr_date.extend([pl.min(col).alias(f"min_{col}") for col in cols])

        return expr_date

    @staticmethod
    def str_expr(df):
        cols = [col for col in df.columns if col[-1] in ("M",)]
        cols.remove('WEEK_NUM')

        expr_str = []
        expr_str.extend([pl.last(col).alias(f"last_{col}") for col in cols])
        expr_str.extend([pl.first(col).alias(f"first_{col}") for col in cols])
        expr_str.extend([pl.n_unique(col).alias(f"n_unique_{col}") for col in cols])

        return expr_str

    @staticmethod
    def other_expr(df):
        cols = [col for col in df.columns if col[-1] in ("T", "L")]

        expr_other = []
        expr_other.extend([pl.max(col).alias(f"max_{col}") for col in cols])
        expr_other.extend([pl.min(col).alias(f"min_{col}") for col in cols])
        expr_other.extend([pl.mean(col).alias(f"mean_{col}") for col in cols])
        expr_other.extend([pl.var(col).alias(f"var_{col}") for col in cols])
        expr_other.extend([pl.last(col).alias(f"last_{col}") for col in cols])
        expr_other.extend([pl.first(col).alias(f"first_{col}") for col in cols])

        return expr_other

    @staticmethod
    def count_expr(df):
        cols = [col for col in df.columns if "num_group" in col]

        expr_count = [pl.max(col).alias(f"max_{col}") for col in cols]

        return expr_count

    @staticmethod
    def get_exprs(df):
        exprs = Aggregator.num_expr(df) + \
                Aggregator.date_expr(df) + \
                Aggregator.str_expr(df) + \
                Aggregator.other_expr(df) + \
                Aggregator.count_expr(df)

        return exprs

In [28]:
fea_agg = data.group_by("case_id").agg(Aggregator.get_exprs(data))

In [29]:
fea_agg.shape

(1526669, 14)

In [30]:
fea_agg.head(20)

case_id,mean_amount_416A,mean_contractenddate_991D_minus_openingdate_313D_P,var_amount_416A,var_contractenddate_991D_minus_openingdate_313D_P,last_amount_416A,last_contractenddate_991D_minus_openingdate_313D_P,first_amount_416A,first_contractenddate_991D_minus_openingdate_313D_P,max_contractenddate_991D,max_openingdate_313D,min_contractenddate_991D,min_openingdate_313D,max_num_group1
i64,f64,f64,f64,f64,f64,i64,f64,i64,date,date,date,date,i64
0,null,null,0.0,0.0,null,null,null,null,null,null,null,null,null
1,null,null,0.0,0.0,null,null,null,null,null,null,null,null,null
2,null,null,0.0,0.0,null,null,null,null,null,null,null,null,null
3,null,null,0.0,0.0,null,null,null,null,null,null,null,null,null
4,null,null,0.0,0.0,null,null,null,null,null,null,null,null,null
5,null,null,0.0,0.0,null,null,null,null,null,null,null,null,null
6,null,null,0.0,0.0,null,null,null,null,null,null,null,null,null
7,null,null,0.0,0.0,null,null,null,null,null,null,null,null,null
8,null,null,0.0,0.0,null,null,null,null,null,null,null,null,null


In [31]:
data = data_base.join(fea_agg, how="left", on="case_id")

In [32]:
data.shape

(1526669, 19)

In [34]:
data.filter(pl.col('max_num_group1').is_not_null())

case_id,date_decision,MONTH,WEEK_NUM,target,IS_TRAIN,mean_amount_416A,mean_contractenddate_991D_minus_openingdate_313D_P,var_amount_416A,var_contractenddate_991D_minus_openingdate_313D_P,last_amount_416A,last_contractenddate_991D_minus_openingdate_313D_P,first_amount_416A,first_contractenddate_991D_minus_openingdate_313D_P,max_contractenddate_991D,max_openingdate_313D,min_contractenddate_991D,min_openingdate_313D,max_num_group1
i64,date,i64,i64,i64,i32,f64,f64,f64,f64,f64,i64,f64,i64,date,date,date,date,i64
225,2019-01-04,201901,0,0,1,0.0,null,0.0,0.0,0.0,null,0.0,null,null,2016-08-16,null,2016-08-16,0
331,2019-01-05,201901,0,0,1,260.374,1095.0,0.0,0.0,260.374,1095,260.374,1095,2018-03-18,2015-03-19,2018-03-18,2015-03-19,0
358,2019-01-05,201901,0,0,1,0.0,null,0.0,0.0,0.0,null,0.0,null,null,2014-09-02,null,2014-09-02,0
390,2019-01-07,201901,0,0,1,70725.27067,912.5,1.4916e10,66612.5,223.68001,null,211748.53,1095,2017-09-30,2016-06-08,2017-07-22,2014-07-23,2
445,2019-01-08,201901,1,0,1,4747.1876,760.5,1.1268e8,1860.5,23735.938,null,0.0,null,2018-02-08,2016-07-19,2017-07-31,2015-04-27,4
450,2019-01-08,201901,1,0,1,0.0,null,0.0,0.0,0.0,null,0.0,null,null,2015-07-08,null,2015-07-08,0
453,2019-01-08,201901,1,0,1,30154.398333,null,2.7279e9,null,90463.195,null,0.0,null,null,2016-03-03,null,2014-06-21,2
582,2019-01-09,201901,1,0,1,827.77405,1644.0,0.0,0.0,827.77405,1644,827.77405,1644,2018-10-02,2014-04-02,2018-10-02,2014-04-02,0
649,2019-01-09,201901,1,0,1,0.0,null,0.0,0.0,0.0,null,0.0,null,null,2014-04-21,null,2014-04-21,0


In [35]:
def filter_cols(df):
    dt_train = df.filter(pl.col('IS_TRAIN') == 1)
    for col in df.columns:
        if col not in ["case_id", "WEEK_NUM", "date_decision", "MONTH", "IS_TRAIN", "target"]:
            isnull = dt_train[col].is_null().mean()

            if isnull > 0.95:
                print(f'col [{col}] to be null dropped...')
                num_positive = dt_train.filter(pl.col(col).is_not_null() & pl.col("target") == 1).shape[0]
                if num_positive < 1000:
                    df = df.drop(col)
                    print(f'col [{col}] null dropped...')

    for col in df.columns:
        if (col not in ["case_id", "WEEK_NUM", "date_decision", "MONTH", "IS_TRAIN", "target"]) & (df[col].dtype == pl.String):
            freq = dt_train[col].n_unique()

            if freq == 1:
                df = df.drop(col)
                print(f'col [{col}] freq dropped...')

    return df

In [36]:
data = data.pipe(filter_cols)

col [mean_contractenddate_991D_minus_openingdate_313D_P] to be null dropped...
col [last_contractenddate_991D_minus_openingdate_313D_P] to be null dropped...
col [first_contractenddate_991D_minus_openingdate_313D_P] to be null dropped...
col [max_contractenddate_991D] to be null dropped...
col [min_contractenddate_991D] to be null dropped...


In [37]:
data.shape

(1526669, 19)

In [38]:
def handle_date(df):
    for col in df.columns:
        if col[-1] in ("D",) and df[col].dtype == pl.Date:
            print(f'col [{col}] is date...')
            df = df.with_columns(pl.col(col) - pl.col("date_decision"))
            df = df.with_columns(pl.col(col).dt.total_days())

    # df = df.drop("date_decision", "MONTH")
    return df


def handle_category(df):
    dt_train = df.filter(pl.col('IS_TRAIN') == 1)
    cols_list = df.columns
    for col in cols_list:
        if ((col[-1] in ("M",)) and ('WEEK_NUM' not in col) and (not col.startswith('n_unique'))) \
                or (col[-1] in ("L", "P") and (df[col].dtype == pl.String)):
            freq = dt_train[col].n_unique()
            if freq <= 10:
                print(f'col [{col}] one hot encoded...')
                encoder = OneHotEncoder(cols=[col], handle_unknown='indicator', handle_missing='indicator', use_cat_names=True)
                encoder.fit(dt_train[col].to_pandas())
                res_df = pl.from_pandas(encoder.transform(df[col].to_pandas()))
                df = pl.concat([df, res_df], how="horizontal")
                df = df.drop(col)
            else:
                print(f'col [{col}] target encoded...')
                encoder = TargetEncoder(cols=[col], handle_unknown='value', handle_missing='value') \
                            .fit(dt_train[col].to_pandas(), dt_train['target'].to_pandas())
                df = df.with_columns(pl.from_pandas(encoder.transform(df[col].to_pandas())).to_series().alias(col + '_target_encoder'))
                df = df.drop(col)

    return df

In [39]:
data = data.pipe(handle_category)

In [40]:
data = data.pipe(handle_date)

col [max_contractenddate_991D] is date...
col [max_openingdate_313D] is date...
col [min_contractenddate_991D] is date...
col [min_openingdate_313D] is date...


In [44]:
data.head(10)

case_id,IS_TRAIN,mean_amount_416A,mean_contractenddate_991D_minus_openingdate_313D_P,var_amount_416A,var_contractenddate_991D_minus_openingdate_313D_P,last_amount_416A,last_contractenddate_991D_minus_openingdate_313D_P,first_amount_416A,first_contractenddate_991D_minus_openingdate_313D_P,max_contractenddate_991D,max_openingdate_313D,min_contractenddate_991D,min_openingdate_313D,max_num_group1
i64,i32,f64,f64,f64,f64,f64,i64,f64,i64,i64,i64,i64,i64,i64
0,1,null,null,0.0,0.0,null,null,null,null,null,null,null,null,null
1,1,null,null,0.0,0.0,null,null,null,null,null,null,null,null,null
2,1,null,null,0.0,0.0,null,null,null,null,null,null,null,null,null
3,1,null,null,0.0,0.0,null,null,null,null,null,null,null,null,null
4,1,null,null,0.0,0.0,null,null,null,null,null,null,null,null,null
5,1,null,null,0.0,0.0,null,null,null,null,null,null,null,null,null
6,1,null,null,0.0,0.0,null,null,null,null,null,null,null,null,null
7,1,null,null,0.0,0.0,null,null,null,null,null,null,null,null,null
8,1,null,null,0.0,0.0,null,null,null,null,null,null,null,null,null


In [42]:
data = data.drop('date_decision', 'MONTH', 'WEEK_NUM', 'target')

In [45]:
data.shape

(1526669, 15)

In [46]:
preprocess_dir_path = "/Users/sophie/workspace/data/kaggle/home-credit-credit-risk-model-stability/preprocess"
preprocess_file = 'deposit_1.parquet'
preprocess_file_path = Path(os.path.join(preprocess_dir_path, preprocess_file))
data.write_parquet(preprocess_file_path)